## Official documentation

## Quick tuturial

In [3]:
import os
# read from the env file
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = (
    {"topic": RunnablePassthrough()} |
    prompt |
    model |
    output_parser
)

chain.invoke({"topic": "ice cream"})

"Why don't ice creams ever get invited to parties?\n\nBecause they always bring the scoop!"

In [9]:
# asychronous
chain.ainvoke("ice cream")

<coroutine object RunnableSequence.ainvoke at 0x7f50b94f8eb0>

In [5]:
# batch
chain.batch(["ice cream", "spaghetti", "dumplings"])

["Why don't ice creams ever get into arguments?\n\nBecause they always stay cool!",
 "Why don't spaghetti like to play hide and seek?\n\nBecause they always pasta their hiding place!",
 "Why don't dumplings ever play hide and seek?\n\nBecause they always end up in hot water!"]

In [10]:
# stream
chain.stream("ice cream")
for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

Why don't ice creams ever get into arguments?

Because they always stay cool!

Below are the internal workflow flow (prompt->model->output_parser) when chain.invoke({"topic": "ice cream"}) is called:

prompt: BasePromptTemplate.invoke(input, config): This method is called first, with input being {"topic": "ice cream"}. It formats the prompt using the input variables and returns a PromptValue object. This object represents a filled prompt that's ready to be passed to a language model.

model: BaseChatModel.invoke(input, config, **kwargs): This method is called next, with input being the PromptValue object returned by the previous step. It converts the PromptValue object into a format that's suitable for the language model, generates a response using the language model, and returns a BaseMessage object. This object represents the response generated by the language model.

output_parser: BaseGenerationOutputParser.invoke(input, config): This method is called last, with input being the BaseMessage object returned by the previous step. It parses the BaseMessage object and returns the parsed result.

Each invoke method is responsible for handling one step in the pipeline, and the output of each step is passed as the input to the next step. This allows for a clear separation of responsibilities and makes it easy to modify or replace individual steps in the pipeline.


# [MermaidChart: 15383246-018f-44ee-9732-71ffd9e6c924]

In [10]:
prompt_value = prompt.invoke({"topic": "ice cream"})
print('prompt_value: ', prompt_value)
print('prompt_value.to_string(): ', prompt_value.to_string())
print('prompt_value.to_messages(): ', prompt_value.to_messages())
print('prompt_value.to_json(): ', prompt_value.to_json())

prompt_value:  messages=[HumanMessage(content='tell me a short joke about ice cream')]
prompt_value.to_string():  Human: tell me a short joke about ice cream
prompt_value.to_messages():  [HumanMessage(content='tell me a short joke about ice cream')]
prompt_value.to_json():  {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'prompts', 'chat', 'ChatPromptValue'], 'kwargs': {'messages': [HumanMessage(content='tell me a short joke about ice cream')]}}


In [18]:
# from langchain_openai import ChatOpenAI
# model = ChatOpenAI(model="gpt-4")
message  = model.invoke(prompt_value)
print('ChatOpenAI message: ', message)

from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
print('OpenAI message: ', llm.invoke(prompt_value))

ChatOpenAI message:  content="Why don't ice creams ever get invited to parties?\n\nBecause they always drip when they're in the heat!"
OpenAI message:  

Robot: Why did the ice cream go to therapy? Because it was having a meltdown!


In [21]:
result = output_parser.invoke(message)
print('result: ', result)

result:  Why don't ice creams ever get invited to parties?

Because they always drip when they're in the heat!


### Add with retrieval component

In [4]:
# Requires:
# pip install langchain docarray tiktoken
# Fix the error in community lib according to PR: https://github.com/langchain-ai/langchain/pull/18678 befor executing this code
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

/home/ubuntu/anaconda3/envs/py310/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


'Harrison worked at Kensho.'

In [6]:
retriever.invoke("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

### Add configurable component

In [16]:
from langchain_core.runnables import ConfigurableField

from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")

configurable_model = model.configurable_alternatives(
    ConfigurableField(id="model"), 
    default_key="chat_openai", 
    openai=llm,
    # anthropic=anthropic,
)
configurable_chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | configurable_model 
    | output_parser
)

In [18]:
stream = configurable_chain.stream(
    "ice cream", 
    config={"model": "openai"}
)
for chunk in stream:
    print(chunk, end="", flush=True)

Why don't ice creams ever get invited to parties?

Because they always melt under pressure!

### Full process with fallback && logging component

In [20]:
import os

from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, ConfigurableField

os.environ["LANGCHAIN_API_KEY"] = "..."
os.environ["LANGCHAIN_TRACING_V2"] = "true"

prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
chat_openai = ChatOpenAI(model="gpt-3.5-turbo")
openai = OpenAI(model="gpt-3.5-turbo-instruct")
anthropic = ChatAnthropic(model="claude-2")
model = (
    chat_openai
    .with_fallbacks([anthropic])
    .configurable_alternatives(
        ConfigurableField(id="model"),
        default_key="chat_openai",
        openai=openai,
        anthropic=anthropic,
    )
)

chain = (
    {"topic": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [21]:
chain.invoke({"topic": "ice cream"})

Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')


'Why did the ice cream truck break down? It had too many "scoops"!'

Failed to batch ingest runs: LangSmithAuthError('Authentication failed for https://api.smith.langchain.com/runs/batch. HTTPError(\'401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/batch\', \'{"detail":"Invalid API key"}\')')


### The built-in optimizations

- Stream: By default, stream runs invoke() in a loop, yielding results.
             Override to optimize streaming.

- Batch: By default, batch runs invoke() in parallel using a thread pool executor.
             Override to optimize batching.

- Async: Methods with "a" suffix are asynchronous. By default, they execute
             the sync counterpart using asyncio's thread pool.
             Override for native async.

### Runnable component

In [1]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

runnable.invoke(1) # {'origin': 1, 'modified': 2}

{'origin': 1, 'modified': 2}

In LangChain Expression Language (LCEL), RunnableParallel, RunnableLambda, and RunnablePassthrough are key components that facilitate the construction of complex chains for processing data, executing custom functions, and managing data flow. Each serves a distinct purpose and operates differently within the LCEL framework. Understanding their usage and differences is crucial for effectively leveraging LangChain for various tasks.

**RunnableParallel**

RunnableParallel allows for the execution of multiple Runnables in parallel, combining their outputs into a single map. This is particularly useful when you need to perform several operations that do not depend on each other and can be executed simultaneously, thereby reducing the overall execution time. It is ideal for scenarios where you need to fetch or process data from multiple sources concurrently

In [4]:
from langchain_core.runnables import RunnableParallel
# Assuming joke_chain and poem_chain are predefined Runnables
joke_chain = RunnableParallel()
poem_chain = RunnableParallel()
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)
result = map_chain.invoke({"topic": "bear"})
print(result["joke"])

{}


**RunnableLambda**

RunnableLambda turns custom Python functions into Runnables that can be integrated into LCEL chains. This component is essential for incorporating arbitrary logic or transformations that are not directly supported by other Runnables. It accepts a single argument, making it necessary to wrap functions that require multiple inputs

In [2]:
from langchain_core.runnables import RunnableLambda
def add_one(x):
    return x + 1
add_one_runnable = RunnableLambda(add_one)
result = add_one_runnable.invoke(5)
print(result) # 6

6


**RunnablePassthrough**

RunnablePassthrough is used to pass inputs through unchanged or with the addition of extra keys. It is particularly useful in scenarios where part of the input needs to be directly forwarded to subsequent components in the chain without modification. It can also be used to assign new keys to the data being passed through

In [6]:
from langchain_core.runnables import RunnablePassthrough
runnable = RunnablePassthrough.assign(extra=lambda x: x["num"] * 3)
result = runnable.invoke({"num": 1})
print(result) # 3

{'num': 1, 'extra': 3}


**Differences**

- Execution Mode: RunnableParallel executes multiple Runnables in parallel and combines their outputs, whereas RunnableLambda and RunnablePassthrough operate on a single input.
- Purpose: RunnableLambda is designed for integrating custom functions into LCEL chains, allowing for arbitrary logic to be executed. In contrast, RunnablePassthrough is used for forwarding inputs unchanged or with minor modifications, and RunnableParallel is for parallel execution of independent tasks.
- Input Handling: RunnableLambda requires a single argument for its custom function, which may necessitate wrapping for multi-argument functions. RunnablePassthrough and RunnableParallel deal with inputs in the form of maps, facilitating the handling of multiple data points simultaneously.